In [1]:
import numpy as np
from PIL import Image
from PIL import ImageOps
import sys
import os
import requests
from io import BytesIO

from keras.models import load_model

In [2]:
def image_to_array(image_path):

    im = Image.open(image_path)
    return np.array(im, dtype=float)

In [3]:
def label_decoder(key):
    label_mapping = {
        "1" : "aca",
        "2" : "n",
        "3" : "scc",
    }
    return label_mapping[key]

In [4]:
def resize(image_name, dirPath):
    
    image_path = folderPath + image_name
    im = Image.open(image_path)
        
    desired_size = 128
    old_size = im.size  # old_size[0] is in (width, height) format

    ratio = float(desired_size)/max(old_size)
    new_size = tuple([int(x*ratio) for x in old_size])

    im = im.resize(new_size, Image.ANTIALIAS)
    
    # create a new image and paste the resized on it
    new_im = Image.new("RGB", (desired_size, desired_size))
    new_im.paste(im, ((desired_size-new_size[0])//2,
                        (desired_size-new_size[1])//2))

    delta_w = desired_size - new_size[0]
    delta_h = desired_size - new_size[1]
    padding = (delta_w//2, delta_h//2, delta_w-(delta_w//2), delta_h-(delta_h//2))
    new_im = ImageOps.expand(im, padding)

    filename, file_extension = os.path.splitext(image_name)
    new_filename = filename + "_resized.jpeg"
    new_im.save(folderPath + new_filename, "JPEG")
    
    return new_filename

In [5]:
def equalize_image(image_name, dirPath): 
    
    image_path = folderPath+ image_name
    im = Image.open(image_path)
    

    im_out = ImageOps.equalize(im)
    
    filename, file_extension = os.path.splitext(image_name) 
    new_filename = filename + "_equalized.jpeg"
    im_out.save(folderPath + new_filename, "JPEG")
    
    return new_filename

In [6]:
def run_model(data):

    
    loaded_model = load_model('gear_classifier_2.model')

    result_array = loaded_model.predict(data).tolist()[0]
    
    index = result_array.index(max(result_array)) + 1
    
    predicted_label = label_decoder(str(index))
    
    return predicted_label

In [7]:
    from keras.utils import normalize
    import tensorflow as tf

    preprocessed_img_path = '/Users/barnatasa/Desktop/Màster_Bioinformàtica/TFM/lung_colon_image_set/train/n/lungn2_resized_equalized.jpeg'
    numpy_data = image_to_array(preprocessed_img_path)
    numpy_data = numpy_data.reshape(1, 3, 128, 128)
    numpy_data = numpy_data.astype('float32')
    numpy_data = normalize(numpy_data)
    result_label = run_model(numpy_data)
    model = load_model('gear_classifier_2.model')
    predictions = model.predict(numpy_data)
    score = tf.nn.softmax(predictions)
    message = "Esta imagen pertenece a un tumor del tipo {} con una precisión de {:.2f} sobre 100."
    print(message.format(result_label, round(100 * np.max(score))))


Esta imagen pertenece a un tumor del tipo n con una precisión de 58.00 sobre 100.
